# <center>Compose Music Using a GRU

### Import Libraries

In [0]:
import tensorflow as tf
from music21 import converter, instrument, note, chord,stream
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import GRU,Convolution1D,Convolution2D, Flatten, Dropout, Dense
from keras import utils as np_utils
from tensorflow.keras import layers

### Create the file path to your folder

In [0]:
os.chdir("//Users//vinitasilaparasetty//Downloads//Project 1- GRU//Input//music_files")#enter the file path from your system.

#### Declare and Initialize variables

In [0]:
musical_note = []
offset = []
instrumentlist=[]

### Load Data

In [0]:
filenames = random.sample(os.listdir('D:\\Proj\\vinita\\Project 1- GRU\Input\\music_files\\'), 5) #Only 5 files are taken at random
musiclist = os.listdir('D:\\Proj\\vinita\\Project 1- GRU\Input\\music_files\\')

### Feature Extraction

In [0]:
for file in filenames:
    matching = [s for s in musiclist if file.split('_')[0] in s]
    print(matching)
    r1 = matching[random.randint(1, len(matching))] 
    string_midi = converter.parse(r1)
    parsednotes = None
    parts = instrument.partitionByInstrument(string_midi)
    instrumentlist.append(parts.parts[0].getInstrument().instrumentName)
    if parts: # file has instrument parts
        parsednotes = parts.parts[0].recurse()
    else: # file has flat notes
        parsednotes = string_midi.flat.notes
    for element in parsednotes: #detect offsets
        offset.append(element.offset)
        if isinstance(element, note.Note):
            musical_note.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            musical_note.append('.'.join(str(n) for n in element.normalOrder))

### Exploratory Data Analysis (EDA)

#### Number of instruments found in music files

In [0]:
pd.Series(instrumentlist).value_counts() 

#### Number of notes and chords

In [0]:
pd.Series(musical_note).value_counts() 

#### Visualisation of offset in the music file

In [0]:
offset = [float(item) for item in offset] 
plt.plot(offset)
plt.show() # this shows that offset is normally started from 0 for each musical file

### Data Preperation

In [0]:
sequence_length = 100

# Arranging notes and chords in ascending order
pitchcategory = sorted(set(item for item in musical_note))

# One hot encoding
note_encoding = dict((note, number) for number, note in enumerate(pitchcategory))
model_input_original = []
model_output = []

# Prepare input and output data for model
for i in range(0, len(musical_note) - sequence_length, 1):
    sequence_in = musical_note[i:i + sequence_length]
    sequence_out = musical_note[i + sequence_length]
    model_input_original.append([note_encoding[char] for char in sequence_in])
    model_output.append(note_encoding[sequence_out])

n_patterns = len(model_input_original)

# converting data for compatibility with GRU
model_input = np.reshape(model_input_original, (n_patterns, sequence_length, 1))

# standardizing model input data
model_output = np_utils.to_categorical(model_output)
Len_Notes = model_output.shape[1]
model_input = model_input / float(Len_Notes)

### Structure of the Model

In [0]:
model_GRU = tf.keras.models.Sequential()
model_GRU.add(layers.GRU(16,input_shape=(model_input.shape[1], model_input.shape[2]),return_sequences=True))
model_GRU.add(layers.Dropout(0.3))
model_GRU.add(layers.GRU(64, return_sequences=True))
model_GRU.add(layers.Dropout(0.3))
model_GRU.add(layers.GRU(64))
model_GRU.add(layers.Dense(16))
model_GRU.add(layers.Dropout(0.3))
model_GRU.add(layers.Dense(Len_Notes))
model_GRU.add(layers.Activation('softmax'))
model_GRU.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model_GRU.summary() #Displays model architecture

### Train Model

In [0]:
# initializing data for model prediction
int_to_note = dict((number, note) for number, note in enumerate(pitchcategory))
pattern = model_input_original[0]
prediction_output = []
model_GRU.fit(model_input, model_output, epochs=30, batch_size=64)

### Prediction

### Generate Input Sequence

In [0]:
# generate 500 notes

for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(Len_Notes)
    prediction_GRU = model_GRU.predict(prediction_input, verbose=0)
    index_GRU = np.argmax(prediction_GRU)
    index = index_GRU
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern,index)
    pattern = pattern[1:len(pattern)]


### Data Preperation

In [0]:
# prepare notes , chords and offset seperately 
offlen = len(offset)
DifferentialOffset = (max(offset)-min(offset))/len(offset)
offset2 = offset.copy()
output_notes = []
i = 0
offset = []
initial = 0
for i in range(len(offset2)):
    offset.append(initial)
    initial  = initial+DifferentialOffset
# Differentiate notes and chords
i=0
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for check_note in notes_in_chord:
            gen_note = note.Note(int(check_note))
            gen_note.storedInstrument = instrument.Guitar()
            notes.append(gen_note)
        gen_chord = chord.Chord(notes)
        gen_chord.offset = offset[i]
        output_notes.append(gen_chord)
    else:
        gen_note = note.Note(pattern)
        gen_note.offset = offset[i]
        gen_note.storedInstrument = instrument.Guitar()
        output_notes.append(gen_note)
    i=i+1

### Create MIDI file

In [0]:
os.chdir('D:\\Proj\\vinita\\Project 1- GRU\Output\\') #Specify file path to store the MIDI file.
midi_stream = stream.Stream(output_notes) #create stream
midi_stream.write('midi', fp='GRU_output.mid') #create MIDI file using stream 